## Training Simba model

Notebook to detail steps of training/fine-tuning/inference of the model used for Simba

In [14]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
import os

/home/freya/simplification/simp_env/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:
dataset = load_dataset('json', data_files='train_inst_dataset.jsonl')
dataset = dataset['train']
dataset.set_format("torch", device="cuda")

**Training datapoint example**

{"original": "Kannst du folgenden Zeitungsartikel vereinfachen: 'Ein Netzwerkfehler sorgte Dienstagvormittag für den Ausfall von 120 der 275 Lifte in den Wiener U-Bahn-Stationen. Die Störung sei in der Nacht auf Dienstag um 3.47 Uhr eingetreten, berichtete ORF Wien . Durch den Netzwerkfehler ist wenig später in den betroffenen Liften die Notruffunktion ausgefallen. Aus Sicherheitsgründen habe man die Aufzüge gestoppt, sagte eine Sprecherin der Wiener Linien. Nur jene Stationen, die mit Aufsichtspersonal besetzt sind, seien nicht betroffen, da dort keine Gefahr bestehe, dass Personen unbemerkt stecken blieben bzw. zu Schaden kämen, hieß es. Nach der Reparatur des Systems wurden die Aufzüge schrittweise wieder in Gang gesetzt. Um 10.30 Uhr waren laut Wiener Linien alle ausgefallenen Aufzüge wieder in Betrieb. Was zu dem Defekt führte, ist derzeit noch Gegenstand von Untersuchungen. Für die Fahrgäste war die Sperre zwar ärgerlich, besondere Vorfälle gab es aber nicht, bestätigte auch die Wiener Rettung auf Nachfrage des KURIER. Der nächste Ausfall ist geplant: Ab 30. April wird die U4 von Hütteldorf bis Hietzing gesperrt .'", "simplification": "In den Wiener U-Bahn-Stationen ist von 18. auf 19. April ein technischer-Fehler aufgetreten. Durch diesen Fehler sind in den betroffenen Aufzügen die Notrufsignale ausgefallen. Aus Sicherheitsgründen waren viele Aufzüge außer Betrieb. Das war in Stationen, wo es keine Aufsichts-Person gab. In den Stationen, wo es eine Aufsicht gibt, wurden keine Aufzüge gestoppt, da keine Gefahr für die Menschen war. Nach der Reparatur von dem technischen Problem wurden schrittweise die Aufzüge wieder in Betrieb genommen. Was der Auslöser für diesen Fehler war, wird noch untersucht. Für die Menschen war die Sperre zwar ärgerlich, aber zum Glück gab es keine Verletzten."}

In [2]:
base_model_name = "jphme/em_german_leo_mistral"
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    #quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map=device_map,
    #trust_remote_code=True,
    #use_auth_token=True
)
#base_model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.12s/it]


In [5]:
output_dir = "./mistral_fine_tuned_feb24"

training_args = TrainingArguments(output_dir=output_dir[2:], learning_rate=3e-5, warmup_steps=20, lr_scheduler_type="cosine", adam_beta1=0.9, adam_beta2=0.95)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

# More info: https://github.com/huggingface/transformers/pull/24906
#base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    logging_steps=10,
    max_steps=500
    #num_train_epochs=1
)

#(output_dir="mistral_fine_tune_1011", learning_rate=3e-5, warmup_steps=20, lr_scheduler_type="cosine", adam_beta1=0.9, adam_beta2=0.95)
#max_seq_length = 512

#Du bist ein hilfreicher Assistent. USER: <instruction> ASSISTANT:

def formatting_prompts_func(example):
    #print(example)
    output_texts = []
    for i in range(len(example['original'])):
        text = f"Du bist ein hilfreicher Assistent. USER: {example['original'][i]} ASSISTANT: {example['simplification'][i]}"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    #max_seq_length=
    formatting_func=formatting_prompts_func,
    args=training_args,
)



/home/freya/simplification/simp_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [5]:
trainer.train()

Step,Training Loss
10,1.531500
20,1.454800
30,1.416600
40,1.334600
50,1.358100
60,1.386700
70,1.331900
80,1.292300
90,1.269600
100,1.313100


TrainOutput(global_step=500, training_loss=1.2665534152984619, metrics={'train_runtime': 1008.9171, 'train_samples_per_second': 1.982, 'train_steps_per_second': 0.496, 'total_flos': 7.254548169488794e+16, 'train_loss': 1.2665534152984619, 'epoch': 0.21})

In [6]:
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

In [6]:
tokenizer.save_pretrained(output_dir)

('./mistral_fine_tuned_feb24/tokenizer_config.json',
 './mistral_fine_tuned_feb24/special_tokens_map.json',
 './mistral_fine_tuned_feb24/tokenizer.json')

### Merging model

In [8]:
from peft import AutoPeftModelForCausalLM, PeftModel
#import torch
from transformers import AutoModelForCausalLM

In [11]:
adapter_model_name="/home/freya/simplification/mistral_fine_tuned_feb24/final_checkpoint/"
base_model_name = "jphme/em_german_leo_mistral"
model = AutoModelForCausalLM.from_pretrained(base_model_name)#, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)#, #device_map="auto")

model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.94s/it]


In [12]:
model.half()
model.save_pretrained("merged_model_mistral_fine_tuned_feb24")

In [8]:
# Push to HF
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="merged_model_mistral_fine_tuned_feb24",
    repo_id="hiig-piai/simba-v01b_merged",
    repo_type="model",
)


model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]




model-00001-of-00003.safetensors:   0%|          | 16.4k/4.94G [00:00<11:56:35, 115kB/s]

model-00001-of-00003.safetensors:   0%|          | 393k/4.94G [00:00<54:55, 1.50MB/s]   

model-00001-of-00003.safetensors:   0%|          | 655k/4.94G [00:00<47:40, 1.73MB/s]

model-00001-of-00003.safetensors:   0%|          | 1.05M/4.94G [00:00<38:56, 2.11MB/s]

model-00001-of-00003.safetensors:   0%|          | 1.43M/4.94G [00:00<34:27, 2.39MB/s]

model-00001-of-00003.safetensors:   0%|          | 1.82M/4.94G [00:00<31:23, 2.62MB/s]

model-00001-of-00003.safetensors:   0%|          | 2.39M/4.94G [00:00<26:45, 3.08MB/s]

model-00001-of-00003.safetensors:   0%|          | 2.97M/4.94G [00:01<24:22, 3.38MB/s]

model-00001-of-00003.safetensors:   0%|          | 3.54M/4.94G [00:01<22:13, 3.71MB/s]

model-00001-of-00003.safetensors:   0%|          | 4.13M/4.94G [00:01<20:01, 4.11MB/s]

model-00001-of-00003.safetensors:  

CommitInfo(commit_url='https://huggingface.co/hiig-piai/simba-v01b_merged/commit/c736300da864101c4861f7d8337fdcbf5d080257', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c736300da864101c4861f7d8337fdcbf5d080257', pr_url=None, pr_revision=None, pr_num=None)

### Inference

In [1]:
prompt_format = "Du bist ein hilfreicher Assistent. USER: Kannst du folgenden Zeitungsartikel vereinfachen: '" 
prompt_format2 = "' ASSISTANT: "

In [2]:
text5 = """Schulen, Kitas, Bürgerämter: Am Mittwoch dürften viele Berlinerinnen und Berliner Einschränkungen bemerken - wegen eines Warnstreiks der Beschäftigten des öffentlichen Dienstes. Gewerkschaften fordern 10,5 Prozent mehr Lohn.

Notbetreuung in Schulen, geschlossene Kitas, eingeschränkte Besetzung der Bürgerämter: Am Mittwoch hat ein Warnstreik der Berliner Beschäftigten, die unter den Tarifvertrag der Länder fallen, begonnen. Es sei mit Unterrichtsausfall an zahlreichen Schulen zu rechnen, sagte ein Sprecher der beteiligten Gewerkschaft Erziehung und Wissenschaft (GEW) am Morgen. Mindestens 100 Kitas blieben geschlossen, sagte ein Verdi-Sprecher dem rbb.

Zum Warnstreik aufgerufen sind die Beschäftigten der Senatsverwaltungen und Bezirksämter, die Schulen und Hochschulen, die Polizeidienststellen, die Feuerwehr, die Kitas und die forstwirtschaftlichen Betriebe des Landes Berlin, wie die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau gemeinsam mitteilten.

Begleitet wird der Streik in Berlin von einer Demo am Vormittag. Sie beginnt am Wittenbergplatz und zieht dann bis zum Platz des 18. März. Die Organisatoren rechnen eigenen Angaben zufolge mit Tausenden Teilnehmern. Auch in den anderen Stadtstaaten Bremen und Hamburg sind die Beschäftigten des öffentlichen Diensts zu Arbeitsniederlegungen aufgerufen.
"""

In [3]:
# Cleaning output (this might be able to be done more cleverly, i.e. maybe there's some model params that do the same thing?)
# ensure output is not cut off mid-sentence
# ensure output doesn't have repetitive content

import re

def split_into_sentences(text):
  # from here: https://github.com/brjezierski/scrapers/blob/51da6fa87879217c5676df87a5f28873ee8e0826/preprocess.py#L88C1-L100C19 
 
  sentences = re.split(r"(?<!\w\.\w.)(?<![0-9]\.)(?<![0-9][0-9]\.)(?<![A-Z]\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", text)
  sentences = [s for s in sentences if s]
  return sentences

from difflib import SequenceMatcher as SM

def remove_repetitive_output(raw_output):

    output_sents = split_into_sentences(raw_output.split("ASSISTANT: ")[1])

    outputs = ""
    sentences = list()

    for x in range(1,len(output_sents)):
        if SM(None,output_sents[x-1],output_sents[x]).ratio() < 0.6:
            if output_sents[x-1][-1] == ".":
                if output_sents[x-1].replace("\n", "") not in sentences:
                    outputs += output_sents[x-1].replace("\n", "") + " "
                    sentences.append(output_sents[x-1].replace("\n", ""))
    
    return outputs


In [4]:
# Load model if not already loaded
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model="merged_model_mistral_fine_tuned_feb24"

model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", torch_dtype=torch.bfloat16)

base_model_name = "jphme/em_german_leo_mistral"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/home/freya/simplification/simp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.64s/it]


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

remove_repetitive_output(decoded)

' Am Mittwoch haben in Berlin viele Beschäftigte des öffentlichen Dienstes gestreikt. Das bedeutet, dass sie nicht gearbeitet haben. Die Beschäftigten haben gestreikt, weil sie mehr Geld bekommen wollen. Sie fordern 10,5 Prozent mehr Lohn. Das bedeutet, dass sie 10,5 Prozent mehr Geld bekommen sollen. '

In [9]:
text2 = """Der Klimawandel bedroht einer Studie zufolge die Gesundheit von immer mehr Menschen. 
Demnach waren im vergangenen Jahr 18 Millionen mehr gefährdete Personen Hitzewellen ausgesetzt als 2016. 
Im Vergleich zum Jahr 2000 waren es sogar 157 Millionen mehr. Das berichteten etliche wissenschaftliche Institutionen im Fachmagazin “The Lancet”. 
Als hitzegefährdet gelten in diesem Zusammenhang Menschen, die über 65 Jahre alt sind, in Städten leben oder an Diabetes, einer Herz-Kreislauf-Erkrankung oder chronischen Atemwegsproblemen leiden. 
Dem Klimawandel sind den Autoren zufolge auch deshalb besonders viele Menschen stark ausgesetzt, weil die Temperaturen in dichter besiedelten Regionen besonders stark steigen: um 0,8 Grad Celsius von 1986 bis 2017. 
Im gleichen Zeitraum stieg die weltweite Durchschnittstemperatur lediglich um 0,3 Grad Celsius. Die Forscher erwarten auch eine Ausbreitung tropischer Krankheiten. 
Die Hitze geht oft einher mit der Luftverschmutzung in den Städten. 97 Prozent der untersuchten Städte in Ländern mit niedrigem und mittlerem Einkommensniveau erfüllen die Luftqualitätsrichtlinien der Weltgesundheitsorganisation (WHO) nicht. 
Die Erwärmung führt auch dazu, dass immer mehr Arbeitsstunden hitzebedingt ausfallen. 2017 waren es 153 Milliarden Stunden weltweit, 62 Milliarden mehr als im Jahr 2000. 
Hinzu kommen weitere ökonomische Verluste: Im vergangenen Jahr führten 712 extreme Wetterereignisse zu einem globalen Verlust von 326 Milliarden Dollar (rund 288 Milliarden Euro), fast das Dreifache der Summe von 2016. 
“Die heutigen Veränderungen der Hitzewellen und des Arbeitsvermögens warnen frühzeitig vor den verstärkten und überwältigenden Auswirkungen auf die öffentliche Gesundheit, die zu erwarten sind, wenn die Temperaturen weiter steigen”, wird Hilary Graham von der englischen University of York in einer “Lancet”-Mitteilung zitiert. 
Doch es gebe auch Lichtblicke, schreiben die Autoren. So sei der weltweite Kohleverbrauch seit 2013 gesunken. 
Die Leistung der 2017 errichteten Kraftwerke teilt sich in 157 Gigawatt aus erneuerbaren Energien und 70 Gigawatt aus fossilen Brennstoffen auf. 
“Aufregende Trends in Schlüsselbereichen für die Gesundheit, darunter der Ausstieg aus Kohle, der Einsatz gesünderer, sauberer Verkehrsträger und die Anpassung des Gesundheitssystems, rechtfertigen einen vorsichtigen Optimismus”, schreiben die Forscher. 
Für das Projekt “The Lancet Countdown: Tracking Progress on Health and Climate Change” haben sich untern anderem die Vereinten Nationen und 27 führende Forschungseinrichtungen zusammengetan. 
Das Projekt beruht auf dem Fachwissen von Klimawissenschaftlern, Medizinern, Ökologen, Mathematikern, Geografen, Ingenieuren, Energie-, Lebensmittel-, Vieh- und Verkehrsexperten, Ökonomen, Sozial- und Politikwissenschaftlern sowie Angehörigen von Gesundheitsbehörden."""

In [7]:
inputs = tokenizer(prompt_format + text2 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

remove_repetitive_output(decoded)

'2017 gab es weltweit 18 Millionen mehr Menschen, die von Hitzewellen betroffen waren. Das ist ein Anstieg von 157 Millionen Menschen im Vergleich zu 2000. Das berichteten Wissenschaftler in der Zeitschrift “The Lancet”. '

In [8]:
text2

'Der Klimawandel bedroht einer Studie zufolge die Gesundheit von immer mehr Menschen. \nDemnach waren im vergangenen Jahr 18 Millionen mehr gefährdete Personen Hitzewellen ausgesetzt als 2016. \nIm Vergleich zum Jahr 2000 waren es sogar 157 Millionen mehr. Das berichteten etliche wissenschaftliche Institutionen im Fachmagazin “The Lancet”. \nAls hitzegefährdet gelten in diesem Zusammenhang Menschen, die über 65 Jahre alt sind, in Städten leben oder an Diabetes, einer Herz-Kreislauf-Erkrankung oder chronischen Atemwegsproblemen leiden. \nDem Klimawandel sind den Autoren zufolge auch deshalb besonders viele Menschen stark ausgesetzt, weil die Temperaturen in dichter besiedelten Regionen besonders stark steigen: um 0,8 Grad Celsius von 1986 bis 2017. \nIm gleichen Zeitraum stieg die weltweite Durchschnittstemperatur lediglich um 0,3 Grad Celsius. Die Forscher erwarten auch eine Ausbreitung tropischer Krankheiten. \nDie Hitze geht oft einher mit der Luftverschmutzung in den Städten. 97 Pro

In [10]:
text7 = """Ab Mittwoch will das Bodenpersonal der Lufthansa in einen 27-stündigen Warnstreik treten - auch der BER ist davon betroffen. Das Unternehmen arbeitet an einem Sonderflugplan. Reisende sollen am Montagnachmittag über ihre Flüge informiert werden.

Flugreisende müssen sich am Mittwoch auf Verzögerungen bei der Lufthansa einstellen - auch am Flughafen BER. Dies betreffe auch die Tochterfirmen der Lufthansa - dazu zählen Eurowings, Swiss, Austrian Airlines sowie Brussels Airlines.

Die Gewerkschaft Verdi hat das Bodenpersonal der Lufthansa zu einem ganztägigen Warnstreik aufgerufen, wie ein Verdi-Sprecher am frühen Montagmorgen mitgeteilt hat. Betroffen seien auch die Standorte Frankfurt am Main, München, Hamburg und Düsseldorf. Der Streik soll laut Verdi am Mittwoch um 4 Uhr starten und bis Donnerstag, 7:10 Uhr andauern."""

In [10]:
inputs = tokenizer(prompt_format + text7 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

remove_repetitive_output(decoded)

'27 Stunden lang streiken die Mitarbeiter der Lufthansa. Das Bodenpersonal der Lufthansa streikt. Das Bodenpersonal ist für die Sicherheit am Flughafen zuständig. Es kontrolliert die Gepäckstücke und die Passagiere. Auch die Lufthansa-Tochterfirmen sind betroffen. Das sind die Firmen, die zur Lufthansa gehören. '

### GPTQ quantization

In [1]:
# Load calibration dataset (unseen training examples)

import pickle
formatted_calibrate_ds = pickle.load(open("calibrated_ds.p", "rb"))

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

fine_tuned_model = "merged_model_mistral_fine_tuned_feb24"
base_model_name = "jphme/em_german_leo_mistral"

#Load model on CPU
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model, torch_dtype=torch.float16)#.to(device)



/home/freya/simplification/simp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]


In [4]:
quantizer = GPTQQuantizer(bits=4, group_size=128, dataset=formatted_calibrate_ds[:150], cache_examples_on_gpu=False)#, block_name_to_quantize = "model.decoder.layers", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

Quantizing model.layers blocks : 100%|██████████| 32/32 [18:13<00:00, 34.19s/it]
Found modules on cpu/disk. Using Exllama/Exllamav2 backend requires all the modules to be on GPU. Setting `disable_exllama=True`


In [6]:
model.save_pretrained(fine_tuned_model+"-gptq", safe_serialization=True)
tokenizer.save_pretrained(fine_tuned_model+"-gptq")

('merged_model_mistral_fine_tuned_feb24-gptq/tokenizer_config.json',
 'merged_model_mistral_fine_tuned_feb24-gptq/special_tokens_map.json',
 'merged_model_mistral_fine_tuned_feb24-gptq/tokenizer.json')

In [7]:
# Push to HF
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path=fine_tuned_model+"-gptq",
    repo_id="hiig-piai/simba-v01b_merged_gptq",
    repo_type="model",
)


model.safetensors: 100%|██████████| 4.16G/4.16G [03:33<00:00, 19.5MB/s] 


CommitInfo(commit_url='https://huggingface.co/hiig-piai/simba-v01b_merged_gptq/commit/4613d8656c36a8bb3899d8d68c7b0545f76c771f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4613d8656c36a8bb3899d8d68c7b0545f76c771f', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
# The model produced pretty bad outputs, so experimenting with different parameters
#Default params:
#https://github.com/huggingface/optimum/blob/da6f9e2d9bc57c4a337ac3a30a831f295325a199/optimum/gptq/quantizer.py#L56

#Changed the bits to 8 (from 4), and also increased the size of the calibration dataset

quantizer = GPTQQuantizer(bits=8, group_size=128, dataset=formatted_calibrate_ds[:1000], cache_examples_on_gpu=False)#, block_name_to_quantize = "model.decoder.layers", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

Quantizing model.layers blocks : 100%|██████████| 32/32 [1:21:59<00:00, 153.72s/it]


In [4]:
model.save_pretrained(fine_tuned_model+"-gptq_8bit", safe_serialization=True)
tokenizer.save_pretrained(fine_tuned_model+"-gptq_8bit")

('merged_model_mistral_fine_tuned_feb24-gptq_8bit/tokenizer_config.json',
 'merged_model_mistral_fine_tuned_feb24-gptq_8bit/special_tokens_map.json',
 'merged_model_mistral_fine_tuned_feb24-gptq_8bit/tokenizer.json')

### Inference with GPTQ model

In [4]:
# Load model if not already loaded
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_repo="hiig-piai/simba-v01b-gptq"


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model, device_map="auto")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 7.54MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 3.46MB/s]
special_tokens_map.json: 100%|██████████| 487/487 [00:00<00:00, 2.73MB/s]


In [2]:
prompt_format = "Du bist ein hilfreicher Assistent. USER: Kannst du folgenden Zeitungsartikel vereinfachen: '" 
prompt_format2 = "' ASSISTANT: "

In [3]:
text5 = """Schulen, Kitas, Bürgerämter: Am Mittwoch dürften viele Berlinerinnen und Berliner Einschränkungen bemerken - wegen eines Warnstreiks der Beschäftigten des öffentlichen Dienstes. Gewerkschaften fordern 10,5 Prozent mehr Lohn.

Notbetreuung in Schulen, geschlossene Kitas, eingeschränkte Besetzung der Bürgerämter: Am Mittwoch hat ein Warnstreik der Berliner Beschäftigten, die unter den Tarifvertrag der Länder fallen, begonnen. Es sei mit Unterrichtsausfall an zahlreichen Schulen zu rechnen, sagte ein Sprecher der beteiligten Gewerkschaft Erziehung und Wissenschaft (GEW) am Morgen. Mindestens 100 Kitas blieben geschlossen, sagte ein Verdi-Sprecher dem rbb.

Zum Warnstreik aufgerufen sind die Beschäftigten der Senatsverwaltungen und Bezirksämter, die Schulen und Hochschulen, die Polizeidienststellen, die Feuerwehr, die Kitas und die forstwirtschaftlichen Betriebe des Landes Berlin, wie die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau gemeinsam mitteilten.

Begleitet wird der Streik in Berlin von einer Demo am Vormittag. Sie beginnt am Wittenbergplatz und zieht dann bis zum Platz des 18. März. Die Organisatoren rechnen eigenen Angaben zufolge mit Tausenden Teilnehmern. Auch in den anderen Stadtstaaten Bremen und Hamburg sind die Beschäftigten des öffentlichen Diensts zu Arbeitsniederlegungen aufgerufen.
"""

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

remove_repetitive_output(decoded)

'10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Beschäftigten der Schulen, Kitas, Bürgerämter und anderen öffentlichen Einrichtungen haben am Mittwoch gestreikt. Die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau haben die Beschäftigten des öffentlichen Dienstes zu Arbeitsniederlegungen aufgerufen. '

In [12]:
decoded.split("ASSISTANT")[1]

': 10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Beschäftigten der Schulen, Kitas, Bürgerämter und anderen öffentlichen Einrichtungen haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau haben die Beschäftigten des öffentlichen Dienstes zu Arbeitsniederlegungen aufgerufen. Die Beschäftigten der Schulen, Kitas, Bürgerämter und anderen öffentlichen Einrichtungen haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Gewerkschaften Verdi, Erziehung und Wissenschaft ('

#### Inference with 8bit gptq model

In [1]:
# Load model if not already loaded
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_repo="merged_model_mistral_fine_tuned_feb24-gptq_8bit"
model = AutoModelForCausalLM.from_pretrained(model_repo, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_repo)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


/home/freya/simplification/simp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/freya/simplification/simp_env/lib/python3.10/site-packages/transformers/quantizers/auto.py:147: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be prevail.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
decoded.split("ASSISTANT")[1]

':  Am Mittwoch haben in Berlin viele Beschäftigte des öffentlichen Dienstes gestreikt. Das bedeutet, dass sie nicht gearbeitet haben. Die Beschäftigten haben gestreikt, weil sie mehr Geld bekommen wollen. Sie fordern 10,5 Prozent mehr Lohn. Das bedeutet, dass sie 10,5 Prozent mehr Geld bekommen sollen. \n\nDie Beschäftigten haben gestreikt, weil sie mehr Geld bekommen wollen. Sie fordern 10,5 Prozent mehr Lohn. Das bedeutet, dass sie 10,5 Pro'

In [7]:
#What happens if we remove linebreaks? Training data does not contain linebreaks

inputs = tokenizer(prompt_format + text5.replace("\n", " ") + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [8]:
decoded.split("ASSISTANT")[1]

':  Am Mittwoch haben in Berlin viele Beschäftigte des öffentlichen Dienstes gestreikt. Das bedeutet, dass sie nicht gearbeitet haben.  Beschäftigte des öffentlichen Dienstes sind zum Beispiel Lehrer, Erzieher, Polizisten, Feuerwehrleute und Mitarbeiter von Bürgerämtern.  Die Beschäftigten haben gestreikt, weil sie mehr Geld bekommen wollen. Sie fordern 10,5 Prozent mehr Lohn.  In Berlin sind am Mittwoch viele Schulen und Kitas geschlossen geblieben. In den Schulen gab es'

In [11]:
#text2 text7
inputs = tokenizer(prompt_format + text2.replace("\n", " ") + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
decoded.split("ASSISTANT")[1]

': 2017 gab es weltweit 18 Millionen mehr Menschen, die von Hitzewellen betroffen waren. Das ist ein Anstieg von 157 Millionen Menschen im Vergleich zu 2000. Das berichteten Wissenschaftler in der Zeitschrift “The Lancet”. 2017 gab es in Städten 0,8 Grad Celsius mehr Hitze als 1986. Das ist mehr als die weltweite Durchschnittstemperatur, die nur um 0,3 Grad Celsius gestiegen ist. 2017 gab es in Städten 97 Prozent mehr Luft'

In [18]:
#text2 text7
inputs = tokenizer(prompt_format + text7.replace("\n", " ") + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [19]:
decoded.split("ASSISTANT")[1]

':  Am Mittwoch streiken die Mitarbeiter der Lufthansa. Das Bodenpersonal der Lufthansa streikt. Das Bodenpersonal ist für die Sicherheit am Flughafen zuständig. Es kontrolliert die Gepäckstücke und die Passagiere. Es ist auch für die Sicherheit am Flughafen zuständig. Die Lufthansa hat einen Sonderflugplan. Der Sonderflugplan ist ein Flugplan, der nur für einen Tag gilt'

In [20]:
# Push to HF
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="merged_model_mistral_fine_tuned_feb24-gptq_8bit",
    repo_id="hiig-piai/simba-v01b-gptq-8bit",
    repo_type="model",
)

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 16.4k/5.00G [00:00<13:13:35, 105kB/s]
model-00001-of-00002.safetensors:   0%|          | 246k/5.00G [00:00<1:23:22, 999kB/s]  
model-00001-of-00002.safetensors:   0%|          | 393k/5.00G [00:00<1:36:08, 866kB/s]
model-00001-of-00002.safetensors:   0%|          | 524k/5.00G [00:00<1:35:45, 870kB/s]
model-00001-of-00002.safetensors:   0%|          | 786k/5.00G [00:00<1:25:50, 970kB/s]
model-00001-of-00002.safetensors:   0%|          | 1.03M/5.00G [00:01<1:16:46, 1.08MB/s]
model-00001-of-00002.safetensors:   0%|          | 1.29M/5.00G [00:01<1:10:44, 1.18MB/s]
model-00001-of-00002.safetensors:   0%|          | 1.49M/5.00G [00:01<1:04:12, 1.30MB/s]
model-00001-of-00002.safetensors:   0%|          | 1.93M/5.00G [00:01<57:08, 1.46MB/s]  
model-00001-of-00002.safetensors:   0%|          | 2.26M/5.00G [00:01<50:49, 1.64MB/s]
model-00001-of-00002.safetensors:   0%|

CommitInfo(commit_url='https://huggingface.co/hiig-piai/simba-v01b-gptq-8bit/commit/2cdacfd2389cf1fa5f839a04aa929de3b36e9dc4', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2cdacfd2389cf1fa5f839a04aa929de3b36e9dc4', pr_url=None, pr_revision=None, pr_num=None)

### Generation parameters

In [13]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

- temperature is 1.0, 
- top_k = 50 (no. of highest probability vocab tokens to keep for top-k filtering)
- repetition_penalty = 1.0 (i.e. no penalty)
- no_repeat_ngram_size = 0 (if >0 then ngrams of that size can only occur once)
- renormalize_logits = False (highly recommended to set this to True?!) https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig.renormalize_logits

##### Length of inputs/outputs

In [15]:
# What is the average length of input/output?

dataset = load_dataset('json', data_files='train_inst_dataset.jsonl')
dataset = dataset['train']


In [56]:
cleaned = dict()
cleaned['original'] = list()
cleaned['simplification'] = list()

wrong_examples = 0
original_lengths, original_lengths_tokens = list(), list()
simpl_lengths, simpl_lengths_tokens = list(), list()

for i, elem in enumerate(dataset['original']):
    if i % 100 == 0:
        print(i)
    characters = elem.split("Kannst du folgenden Zeitungsartikel vereinfachen: '")[1]
    if len(characters) < 100:
        #print(i, len(characters))
        wrong_examples += 1
    else:
        cleaned['original'].append(elem)
        cleaned['simplification'].append(dataset['simplification'][i])
        original_lengths.append(len(characters))
        original_lengths_tokens.append(len(characters.split(" "))) #note this is not accurate tokenization, but will do
        characters_simpl = dataset['simplification'][i]
        simpl_lengths.append(len(characters_simpl))
        simpl_lengths_tokens.append(len(characters_simpl.split(" ")))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400


In [14]:
cleaned['original']

NameError: name 'cleaned' is not defined

In [12]:
import json
json.dump(cleaned, open("cleaned.json", "w"))

NameError: name 'cleaned' is not defined

In [57]:
wrong_examples

476

In [59]:
len(cleaned['original'])

8966

In [52]:
print(sum(original_lengths)/len(original_lengths))
#2776 characters
print(sum(original_lengths_tokens)/len(original_lengths_tokens))
#390 tokens

2776.1253624804817
390.76622797233995


In [25]:
print(sum(simpl_lengths)/len(simpl_lengths))
#2636 characters
print(sum(simpl_lengths_tokens)/len(simpl_lengths_tokens))
#371 tokens

810.6872484643084
122.21902139377251


Input has about 370 tokens, output 120

#### Testing parameters

In [27]:
#Changing max_new_tokens to 120, to match training data average

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)



Du bist ein hilfreicher Assistent. USER: Kannst du folgenden Zeitungsartikel vereinfachen: 'Schulen, Kitas, Bürgerämter: Am Mittwoch dürften viele Berlinerinnen und Berliner Einschränkungen bemerken - wegen eines Warnstreiks der Beschäftigten des öffentlichen Dienstes. Gewerkschaften fordern 10,5 Prozent mehr Lohn.

Notbetreuung in Schulen, geschlossene Kitas, eingeschränkte Besetzung der Bürgerämter: Am Mittwoch hat ein Warnstreik der Berliner Beschäftigten, die unter den Tarifvertrag der Länder fallen, begonnen. Es sei mit Unterrichtsausfall an zahlreichen Schulen zu rechnen, sagte ein Sprecher der beteiligten Gewerkschaft Erziehung und Wissenschaft (GEW) am Morgen. Mindestens 100 Kitas blieben geschlossen, sagte ein Verdi-Sprecher dem rbb.

Zum Warnstreik aufgerufen sind die Beschäftigten der Senatsverwaltungen und Bezirksämter, die Schulen und Hochschulen, die Polizeidienststellen, die Feuerwehr, die Kitas und die forstwirtschaftlichen Betriebe des Landes Berlin, wie die Gewerkscha

In [28]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

# Kind of repetitive!

10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Beschäftigten der Schulen, Kitas, Bürgerämter und anderen öffentlichen Einrichtungen haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau haben die Beschäftigten
10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Lohn bekommen. Die Beschäftigten der Schulen, Kitas, Bürgerämter und anderen öffentlichen Einrichtungen haben am Mittwoch gestreikt. 


In [29]:
# Changing repetition penalty to 1.2
#We find that using a greedy sampling and θ ≈ 1.2 yields a good balance between truthful generation and lack of repetition
#https://arxiv.org/pdf/1909.05858.pdf

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.2)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)



In [30]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

# Not correct, random date and also "in ganz Deutschland" when really it's just about Berlin, (and Bremen and Hamburg)
# Also the last sentence "Deshalb werden sie gestreikt" doesn't make sense

24.03.2021 – In ganz Deutschland streiken heute Menschen im öffentlichen Dienst. Das bedeutet: Kindergärtnerinnen, Lehrkräfte oder Krankenschwestern gehen nicht zur Arbeit. Der Grund dafür ist eine Auseinandersetzung zwischen den Gewerkschaften und den Politikern über das Geld für ihre Arbeit. Die Gewerkschaften wollen mehr Geld haben. Deshalb werden sie gestreikt. Für die Eltern heißt das
24.03.2021 – In ganz Deutschland streiken heute Menschen im öffentlichen Dienst. Das bedeutet: Kindergärtnerinnen, Lehrkräfte oder Krankenschwestern gehen nicht zur Arbeit. Der Grund dafür ist eine Auseinandersetzung zwischen den Gewerkschaften und den Politikern über das Geld für ihre Arbeit. Die Gewerkschaften wollen mehr Geld haben. Deshalb werden sie gestreikt. 


In [32]:
#Trying repetition penalty 1.1

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.1)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [33]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

20.000 Beschäftigte im öffentlichen Dienst haben am Mittwoch gestreikt. In Berlin waren es etwa 10.000 Beschäftigte. Der Grund ist, dass sie mehr Geld für ihre Arbeit wollen. Sie forderten 10,5 % mehr Lohn. Das bedeutet, dass sie 10,5 % mehr Geld bekommen sollen. Für die Beschäftigten im öffentlichen Dienst gibt es einen Tarifvertrag. Darin steht, was
20.000 Beschäftigte im öffentlichen Dienst haben am Mittwoch gestreikt. In Berlin waren es etwa 10.000 Beschäftigte. Der Grund ist, dass sie mehr Geld für ihre Arbeit wollen. Sie forderten 10,5 % mehr Lohn. Das bedeutet, dass sie 10,5 % mehr Geld bekommen sollen. Für die Beschäftigten im öffentlichen Dienst gibt es einen Tarifvertrag. 


In [36]:
len(text5.split(" "))

173

In [37]:
#Trying repetition penalty 1.05

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.05)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [38]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

#Kind of good although the 20.000 is not in the original

20.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Geld. Sie fordern 10,5 Prozent mehr Lohn. Das ist eine große Summe. Der öffentliche Dienst ist ein wichtiger Bereich. Dazu gehören zum Beispiel Schulen, Kindergärten, Bürgerämter und Polizei. Die Beschäftigten im öffentlichen Dienst arbeiten für den Staat. Sie werden
20.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Geld. Sie fordern 10,5 Prozent mehr Lohn. Das ist eine große Summe. Der öffentliche Dienst ist ein wichtiger Bereich. Dazu gehören zum Beispiel Schulen, Kindergärten, Bürgerämter und Polizei. Die Beschäftigten im öffentlichen Dienst arbeiten für den Staat. 


In [39]:
#Trying repetition penalty 1.02

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.02)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

#Kind of good although the 10.000 is not in the original and the non postprocessed version is still kind of repetitive?

#--> I think it's better if we do not have a repetition penalty, and just use our postprocessing function. Maybe we can try the ngram penalty parameter...

10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Geld. Sie fordern 10,5 Prozent mehr Lohn. Das ist eine große Summe. Die Beschäftigten arbeiten in Schulen, Kitas, Bürgerämtern und anderen öffentlichen Einrichtungen. Sie wollen mehr Geld für ihre Arbeit. Sie wollen auch mehr Geld für die Ausbildung. Sie wollen mehr Geld für die Fortbild
10.000 Beschäftigte des öffentlichen Dienstes in Berlin haben am Mittwoch gestreikt. Sie wollen mehr Geld. Sie fordern 10,5 Prozent mehr Lohn. Das ist eine große Summe. Die Beschäftigten arbeiten in Schulen, Kitas, Bürgerämtern und anderen öffentlichen Einrichtungen. 


In [41]:
#Trying no repetition penalty, and no_repeat_ngram_size of 4

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=4)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [42]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

# These outputs are so bad... grammatically wrong, factually wrong

10.000 Beschäftigte des öffenlichen Dienst in Berlin haben am Mittwochnachmittag gestreikt. Sie wollen mehr Geld. Sie fordern 5 Prozent Lohnerhöhung. Das sind 10 Prozent mehr als die Arbeitgeber anbieten. Die Arbeitgebers wollen nur 3,5 Prozente mehr Lohn zahlen. Die Beschäftigten im öffentliche Dienst arbeiten in Schulen und Kindergärten. Sie arbeiten auch
10.000 Beschäftigte des öffenlichen Dienst in Berlin haben am Mittwochnachmittag gestreikt. Sie wollen mehr Geld. Sie fordern 5 Prozent Lohnerhöhung. Das sind 10 Prozent mehr als die Arbeitgeber anbieten. Die Arbeitgebers wollen nur 3,5 Prozente mehr Lohn zahlen. Die Beschäftigten im öffentliche Dienst arbeiten in Schulen und Kindergärten. 


In [43]:
#Again no repetition penalty, and no_repeat_ngram_size of 4, this time with renormalize_logits

inputs = tokenizer(prompt_format + text5 + prompt_format2, return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=120, pad_token_id=tokenizer.eos_token_id, no_repeat_ngram_size=4, renormalize_logits=True)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [44]:
print(decoded.split("ASSISTANT: ")[1])
print(remove_repetitive_output(decoded))

# No difference, still awful?!?!?

10.000 Beschäftigte des öffenlichen Dienst in Berlin haben am Mittwochnachmittag gestreikt. Sie wollen mehr Geld. Sie fordern 5 Prozent Lohnerhöhung. Das sind 10 Prozent mehr als die Arbeitgeber anbieten. Die Arbeitgebers wollen nur 3,5 Prozente mehr Lohn zahlen. Die Beschäftigten im öffentliche Dienst arbeiten in Schulen und Kindergärten. Sie arbeiten auch
10.000 Beschäftigte des öffenlichen Dienst in Berlin haben am Mittwochnachmittag gestreikt. Sie wollen mehr Geld. Sie fordern 5 Prozent Lohnerhöhung. Das sind 10 Prozent mehr als die Arbeitgeber anbieten. Die Arbeitgebers wollen nur 3,5 Prozente mehr Lohn zahlen. Die Beschäftigten im öffentliche Dienst arbeiten in Schulen und Kindergärten. 
